# 第三章 TensorFlow入门

In [26]:
import tensorflow as tf

## 3.1 计算图 ——TensorFlow的计算模型
编写TensorFlow程序时，系统会自动维护一个默认的计算图。当我们需要更多的计算图来完成工作的时候，可以通过Graph()函数来生成新的计算图。

In [27]:
a = tf.constant([1.0,2.0],name = 'a')
b = tf.constant([3.0,5.0],name = 'b')
result = a+b
print('result = ',result)
print(a.graph is tf.get_default_graph())
print(a.graph is b.graph)

result =  Tensor("add_5:0", shape=(2,), dtype=float32)
True
True


使用as_default()函数可以将计算图指定为默认的。

In [28]:
g1 = tf.Graph()
with g1.as_default():
    v = tf.get_variable("v", [1], initializer = tf.zeros_initializer()) # 设置初始值为0

g2 = tf.Graph()
with g2.as_default():
    v = tf.get_variable("v", [1], initializer = tf.ones_initializer())  # 设置初始值为1
    
with tf.Session(graph = g1) as sess:
    tf.global_variables_initializer().run()
    with tf.variable_scope("", reuse=True):
        print(sess.run(tf.get_variable("v")))

with tf.Session(graph = g2) as sess:
    tf.global_variables_initializer().run()
    with tf.variable_scope("", reuse=True):
        print(sess.run(tf.get_variable("v")))

[0.]
[1.]


以上代码产生了两个计算图,每个计算图中定义了 一个名字为“ v ”的变量。在计算图gl 中,将 v 初始化为 0 ;在计算图 g2 中,将 v 初始化为 l 。可 以看到当运行不同计算图时,变量 v 的值也是不一样的。 TensorFlow 中的计算图不仅仅可以用来隔离张量和计算,它还提供了管理张量和计算的机制。计算图可以通过 tf. Graph.device 函数来指定运行计算的设备。这为 TensorFlow 使用GPU提供了机制。以下程序可以将加法计算跑在GPU上。

In [29]:
g = tf.Graph()
with g.device('/gpu:0'):
    result = a+b
print(result)

Tensor("add_6:0", shape=(2,), dtype=float32)


在一个计算图中,可以通过集合(collection)来管理不同类别的资源。比如通过tf.add_to_collection函数可以将资源加入一个或多个集合中,然后通过tf.get_collection 获取一个集合里面的所有资源。这里的资源可以是张量、变量或者运行TensorFlow程序所需要的队列资源,等等。为了方便使用,TensorFlow也自动管理了一些最常用的集合,下表总结了最常用的几个自动维护的集合。


集合名称 | 集合内容| 使用场景 
-|-|-
tf.GraphKeys. VARIABLES|所有变量|持久化 TensorFlow 模型
tf.GraphKeys.TRAINABLE_ VARIABLES|可学习的变量(一般指神经网络中的参数)|模型训练、生成模型可视化内容
tf.GraphKeys.SUMMARIES|日志生成相关的张量|TensorFlow计算可视化
tf.GraphKeys.QUEUE_RUNNERS|处理输入的QueueRunner|输入处理
tf.GraphKeys.MOVING_AVERAGE_ VARIABLES|所有计算了滑动平均值的变量|计算变量的滑动平均值

## 3.2 TensorFlow数据模型——张量

In [30]:
import tensorflow as tf
a = tf.constant([1.0, 2.0], name="a")
b = tf.constant([2.0, 3.0], name="b")
result = a + b
print(result)

sess = tf.InteractiveSession()
print(result.eval())
sess.close()

Tensor("add_7:0", shape=(2,), dtype=float32)
[3. 5.]


/home/xpt/anaconda3/lib/python3.7/site-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


## 3.3 TensorFlow运行模型——会话

### 创建和关闭会话

In [31]:
# 创建一个会话。
sess = tf.Session()

# 使用会话得到之前计算的结果。
print(sess.run(result))

# 关闭会话使得本次运行中使用到的资源可以被释放。
sess.close()

[3. 5.]


### 使用with statement 来创建会话

In [32]:
with tf.Session() as sess:
    print(sess.run(result))

[3. 5.]


### 指定默认会话

In [33]:
sess = tf.Session()
with sess.as_default():
     print(result.eval())

[3. 5.]


In [34]:
sess = tf.Session()

# 下面的两个命令有相同的功能。
print(sess.run(result))
print(result.eval(session=sess))

[3. 5.]
[3. 5.]


### 使用tf.InteractiveSession构建会话

In [35]:
sess = tf.InteractiveSession ()
print(result.eval())
sess.close()

[3. 5.]


/home/xpt/anaconda3/lib/python3.7/site-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


### ConfigProto配置会话
allow_soft_placement和log_device_placement是两个比较常用的参数，allow_soft_placement用于在某些条件下将计算从GPU移至CPU进行执行，log_device_placement设置为True时日志中将会记录每个节点被安排在了哪个设备上以方便调试。

In [36]:
config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)
sess1 = tf.InteractiveSession(config=config)
sess2 = tf.Session(config=config)

/home/xpt/anaconda3/lib/python3.7/site-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '
